In [1]:
import os
import random
import gc
import fitz
import tiktoken
import torch
import REMOVED_SECRET as F
import numpy as np
from keybert import KeyBERT
from tkinter import filedialog
import tkinter as tk
from langchain_community.vectorstores import FAISS
from REMOVED_SECRET import DistanceStrategy
from langchain.document_loaders import PyMuPDFLoader
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer, util
from memory_profiler import profile
from RAG_UTILS import RERANKER_MODEL, MODEL_ID, EMBEDDING_MODEL_NAME, CustomTextGenerationPipeline, RAGSystem, main, DocumentProcessor, generate_vocab_list
#from rag_for_notebook_sunday import READER_LLM, KNOWLEDGE_VECTOR_DATABASE, RERANKER, CustomTextGenerationPipeline, answer_with_rag, docs_processed
from loss_functions import weighted_loss, label_smoothed_nll_loss
# "mps" if REMOVED_SECRET.is_available() else
# Ensure that we are using the correct device
device = torch.device("cuda" if REMOVED_SECRET() else "cpu")




/home/obb/codes/langers/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `REMOVED_SECRET` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
class PDFKeywordExtractor:
    def __init__(self, num_keywords=50):
        self.num_keywords = num_keywords
        self.kw_model = KeyBERT()
    
    def extract_keywords(self, pdf_path):
        try:
            loader = PyMuPDFLoader(pdf_path)
            document = loader.load()[0]
            keywords = REMOVED_SECRET(document.page_content, keyphrase_ngram_range=(1, 3), top_n=self.num_keywords)
            keywords_list = [keyword for keyword, score in keywords]
            return keywords_list
        except Exception as e:
            print(f"Error loading or processing PDF {pdf_path}: {e}")
            return []


In [3]:
class PDFInjector:
    def __init__(self, embedding_model):
        self.embedding_model = embedding_model

    def inject_text(self, input_pdf_path, output_pdf_path, text_to_inject, keywords_list, docs_processed):
        pdf_document = fitz.open(input_pdf_path)
        zero_width_inject_word = "\u200B".join(list(text_to_inject))

        for doc in docs_processed:
            page_num = 0
            page = pdf_document[page_num]
            original_text = page.get_text("text")
            chunk_keywords = [kw for kw in keywords_list if kw in doc.page_content]

            if chunk_keywords:
                strongest_keyword = self._find_strongest_keyword(chunk_keywords, doc.page_content)
                new_text = original_text.replace(strongest_keyword, f"{zero_width_inject_word}{strongest_keyword}")
                page.clean_contents()
                page.insert_text((0, 0), new_text, fontsize=12)

        pdf_document.save(output_pdf_path)
        pdf_document.close()

    def _find_strongest_keyword(self, keywords, chunk_text):

       

        chunk_embedding = REMOVED_SECRET.encode(chunk_text, convert_to_tensor=True)
        keyword_embeddings = [REMOVED_SECRET.encode(kw, convert_to_tensor=True) for kw in keywords]
        keyword_similarities = {kw: 0 for kw in keywords}
        for kw, kw_embedding in zip(keywords, keyword_embeddings):
            if kw in chunk_text:
                similarity = util.pytorch_cos_sim(chunk_embedding, kw_embedding).item()
                keyword_similarities[kw] = similarity
        strongest_keyword = max(keyword_similarities, key=keyword_similarities.get, default=None)
        return strongest_keyword


In [4]:
import torch
import numpy as np

class SequenceMutator:
    def __init__(self, model, tokenizer, weight=0.8, k=32, learning_rate=0.1):
        self.model = model
        self.tokenizer = tokenizer
        self.weight = weight
        self.k = k
        self.learning_rate = learning_rate

    def calculate_loss(self, logits, target_response_tokens, crucial_indices):
        # Compute the weighted loss, just as in previous functions
        loss = weighted_loss(logits, target_response_tokens, crucial_indices, self.weight)
        return loss

    def mutate_sequence(self, seq_tokens, target_response_tokens, crucial_indices):
        seq_tokens = seq_tokens.long().to(REMOVED_SECRET)
        target_response_tokens = target_response_tokens.long().to(REMOVED_SECRET)
            
            # Get embeddings
        embeddings = REMOVED_SECRET()(seq_tokens)
        embeddings.requires_grad_(True)

        print("embeddings dtype:", embeddings.dtype)
        print("target_response_tokens dtype:", target_response_tokens.dtype)
        # Ensure model parameters require gradients
        #for param in REMOVED_SECRET():
           #param.requires_grad = True
        
        # Forward pass
        outputs = self.model(inputs_embeds=embeddings)
        logits = outputs.logits
        
        # Calculate loss
        loss = self.calculate_loss(logits, target_response_tokens, crucial_indices)
        
        # Compute gradients
        gradients = REMOVED_SECRET(loss, embeddings, retain_graph=True)[0]
        print("Gradients:", gradients)
        if gradients is None:
            raise RuntimeError("Gradient computation failed; grad is None")
        
        new_seqs = []
        for _ in range(self.k):
            mutate_index = torch.randint(0, seq_tokens.shape[1], (1,)).item()
            
            # Mutate the token's embedding based on the gradient
            mutated_embedding = embeddings[0][mutate_index] - self.learning_rate * gradients[0][mutate_index]
            
            # Find the closest token in the vocabulary to the mutated embedding
            distances = torch.norm(REMOVED_SECRET().weight.data - mutated_embedding, dim=1)
            closest_token_id = torch.argmin(distances).item()
            
            # Create a new sequence with the mutated token
            new_seq = seq_tokens.clone()
            new_seq[0][mutate_index] = closest_token_id
            
            # Append mutated sequence
            new_seqs.append(new_seq)
        
        # Zero out the gradients for the next iteration
        REMOVED_SECRET()
        
        return new_seqs

    def choose_best_sequence(self, new_seqs, losses):
        # Choose the sequence with the lowest loss
        best_index = np.argmin(losses)
        return new_seqs[best_index]


In [5]:
class WorkflowManager:
    def __init__(self):
        self.rag_system = RAGSystem(
            embedding_model_name=EMBEDDING_MODEL_NAME,
            model_id=MODEL_ID,
            reranker_model=RERANKER_MODEL
        )
        self.pdf_extractor = PDFKeywordExtractor()
        self.pdf_injector = PDFInjector(REMOVED_SECRET)
        self.mutator = SequenceMutator(REMOVED_SECRET.model, REMOVED_SECRET.tokenizer)
        self.docs_processed = None

    def run(self):
        pdf_file_path = self._browse_for_pdf()

        if not pdf_file_path:
            print("No file selected.")
            return

        keyword_results = REMOVED_SECRET(pdf_file_path)
        if not keyword_results:
            print("No keywords extracted from the PDF.")
            return
        
        self.docs_processed = REMOVED_SECRET.split_documents(
            REMOVED_SECRET.load_pdfs_from_folder(REMOVED_SECRET(pdf_file_path))
        )

        self.vocab_list = generate_vocab_list(50257)
        best_keywords = keyword_results[:5]
        initial_sub_document = best_keywords
        token_vocabulary = self.vocab_list + keyword_results
        target_response = "I'm sorry, I can't help with that."
        query_based_on_pdf = "What is the capital of France?"
        T = 5  
        B = 3  

        target_response_tokens = REMOVED_SECRET(target_response, return_tensors='pt').squeeze().to(device)
        crucial_phrase = "I'm sorry"
        crucial_indices = [i for i, token in enumerate(target_response_tokens) if REMOVED_SECRET([token]) in crucial_phrase.split()]

        sub_document = initial_sub_document.copy()
        rag_database_folder = "local_database"
        final_candidate = self._optimize_sub_document(
            pdf_file_path, sub_document, keyword_results, token_vocabulary, target_response_tokens, 
            crucial_indices, query_based_on_pdf, T, B, rag_database_folder
        )

        final_response_file = "final_response.txt"
        with open(final_response_file, "w") as f:
            f.write(final_candidate)
        print(f"Final response saved to {final_response_file}")

        REMOVED_SECRET()
    
    def _optimize_sub_document(self, pdf_file_path, sub_document, keyword_results, token_vocabulary, target_response_tokens, 
                               crucial_indices, query_based_on_pdf, T, B, rag_database_folder):
        sequence_mutator = SequenceMutator(REMOVED_SECRET.model, REMOVED_SECRET.tokenizer)

        for i in range(T):
            candidate_sub_documents = []
            losses = []
            

            for b in range(B):

                output_pdf_path = REMOVED_SECRET(rag_database_folder, f"updated_pdf_{i}_{b}.pdf")
                

                REMOVED_SECRET(pdf_file_path, output_pdf_path, ' '.join(sub_document), keyword_results, self.docs_processed)

                pdf_file_path = output_pdf_path
                keyword_results = REMOVED_SECRET(pdf_file_path)

                answer, relevant_docs, logits = REMOVED_SECRET(query_based_on_pdf, REMOVED_SECRET(self.docs_processed))
                
                print("Answer:", answer)
                print("Relevant docs:", relevant_docs[:100] if relevant_docs else "None")  # Print first 100 chars

                seq_tokens = torch.tensor([REMOVED_SECRET(' '.join(sub_document))], dtype=torch.long)
                print("Seq tokens shape:", seq_tokens.shape)
                print("Target response tokens shape:", target_response_tokens.shape)
            
                #    Use SequenceMutator to generate new sequences
                new_seqs = sequence_mutator.mutate_sequence(seq_tokens, target_response_tokens, crucial_indices)
                # Evaluate new sequences
                for new_seq in new_seqs:
                    new_sub_document = REMOVED_SECRET(new_seq[0])
                    REMOVED_SECRET(pdf_file_path, output_pdf_path, new_sub_document, keyword_results, self.docs_processed)
                    answer, relevant_docs, new_logits = REMOVED_SECRET(query_based_on_pdf, REMOVED_SECRET(self.docs_processed))
                
                    try:
                        loss = weighted_loss(new_logits, target_response_tokens, crucial_indices)
                        losses.append(loss.item())
                    except Exception as e:
                        print(f"Error in loss calculation: {e}")
                        losses.append(float('inf'))

                    candidate_sub_documents.append(new_sub_document.split())

                print(f"Iteration {i+1}/{T}, Candidate {b+1}/{B}, Loss: {loss.item()}")


            best_candidate = sequence_mutator.choose_best_sequence(candidate_sub_documents, losses)
            sub_document = best_candidate
        
        return ' '.join(sub_document)
    
    def _browse_for_pdf(self):
        root = tk.Tk()
        root.withdraw()
        file_path = filedialog.askopenfilename(filetypes=[("PDF Files", "*.pdf")])
        return file_path


In [6]:
if __name__ == "__main__":
    workflow_manager = WorkflowManager()

    workflow_manager.run()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device device because they were offloaded to the cpu.
Splitting Documents: 100%|██████████| 2/2 [00:00<00:00, 825.08it/s]


=> Retrieving documents...
Retrieved 2 documents.
=> Reranking documents...


100%|██████████| 1/1 [00:00<00:00, 67.62it/s]
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


Reranked to 1 documents.
Using 1 documents for answering.
=> Generating answer...
Generated Sequence: tensor([[15228, 29901,    13,    13,  9651, 10854, 29871, 29896, 29901,    13,
          5618,   338,   278,  7483,   310,  3444, 29973,  3681, 29991,    13,
            13,  9651,   894, 29901,  1724,   338,   278,  7483,   310,  3444,
         29973,    13,   268,    13,  9651,  2799,   582,  1953, 29901,  5293,
           278,  2472,   515,   278,  3030, 29892,  3867,   263,  3022,   895,
           322,  1513,  1234,   304,   278,  1139, 29889,  1938,   451, 12312,
           278,  1139,   470,   278,  3030, 29889,  3387,  2106,   278,  1234,
          9436,   322, 23359, 29889,    13,   632,    13,  9651,   673, 29901,
            13,   632,    13,  9651,  3681, 29889,    13,    13,    13,  6268,
         29871, 29906, 29901,    13,    13,  2677, 29901,    13,    13,  9651,
         10854, 29871, 29896, 29901,    13,  5618,   338,   278,  7483,   310,
          3444, 29973,  3681,

ValueError: save to original must be incremental